# Day 17
## Part 1

In [19]:
from collections import defaultdict

test_data = ">>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>"

def rocks():
    while True:
        yield [[True, True, True, True]]
        yield [[False, True, False], [True, True, True], [False, True, False]]
        yield [[False, False, True], [False, False, True], [True, True, True]]
        yield [[True], [True], [True], [True]]
        yield [[True, True], [True, True]]
        
def jets(s):
    while True:
        for jet in jets:
            yield jet
            
def draw(chamber, rock = None, rock_x = None, rock_y = None):
    if rock is None:
        for r in reversed(range(min(chamber.keys()), max(chamber.keys()) + 4)):
            print(f"|{''.join('#' if c else '.' for r in chamber[r])}|")
    else:
        rock_height = len(rock)
        rock_width = len(rock[0])
        
    print("+-------+")                    
        
def initial_chamber():
    c = defaultdict(lambda: [False] * 7)
    c[1] = [False] * 7
    return c
                            
def height(chamber):
    for h in reversed(sorted(chamber.keys())):
        if any(chamber[h]):
            return h
    return 0
        

In [20]:
draw(initial_chamber())

|.......|
|.......|
|.......|
|.......|
+-------+


['.', '.', '.', 0, '.', '.', '.']